In [1]:
import os
import sys

sys.path.append("/mnt/lareaulab/reliscu/code")

from parse_gtf import *

In [60]:
psi = pd.read_csv("data/tasic_2018_ALM_STAR_SyntheticDataset1_20pcntCells_25SD_200samples_SJ_pseudobulk_min_observed0.25_minPsi0.1_PSI.csv", index_col=0)
top_qval_mods_df = pd.read_csv("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream_20pcntCells_25SD_200samples_log2_pseudobulk_PosFDR_top_Qval_modules.csv")

### Add gene names to data

In [ ]:
gtf_file = "/mnt/lareaulab/reliscu/data/GENCODE/GRCm39/gencode.vM35.annotation.gtf"
gtf = gtf_parse(gtf_file)
gtf_subset = gtf.loc[gtf['feature'].isin(["gene"])]
attrs = gtf_subset["attribute"].apply(extract_attributes)
attrs_df = attrs.apply(pd.Series)
gtf_parsed = pd.concat([gtf_subset.drop(columns=["attribute"]), attrs_df], axis=1)

In [ ]:
gtf_parsed['gene_id'] = gtf_parsed['gene_id'].str.split(".").str[0]
psi['gene_id'] = psi.index.str.split("_").str[0]
psi['exon_id'] = psi.index.values

In [58]:
psi_anno = pd.merge(gtf_parsed[['gene_id', 'gene_name']], psi, on="gene_id", how="right")
psi_anno = psi_anno.set_index("exon_id").rename_axis(None)
psi_anno = psi_anno.drop(columns=["gene_id"])

### Calc. corr between ME and exon PSI

In [ ]:
# For each cell type module, correlate its eigengene with exon PSI:

eigengene_vs_psi <- lapply(seq_along(filtered_mes$Cell_type), function(i) {
    me <- fread(filtered_mes$ME_path[i], data.table=FALSE)
    me <- me[,grep(paste0("^", filtered_mes$Module[i], "$"), colnames(me))]
    me_psi_cor <- apply(psi[,-1], 1, function(exon) {
        cor_test <- cor.test(exon, me, alternative="greater")
        data.frame(Cor=cor_test$estimate, Pval=cor_test$p.value)
    })
    data.frame(Cell_type=filtered_mes$Cell_type[i], 
               Gene_name=gene_names, 
               Gene_ID=gene_ids, 
               Exon=exon_ids, 
               Exon_Mean_Expr=mean_expr,
               do.call(rbind, me_psi_cor), 
               row.names=NULL)
})
names(eigengene_vs_psi) <- filtered_mes$Cell_type